In [1]:
#
# File: Assignment07_1a.py
# Name: Christopher M. Anderson
# Date: 10/18/2020
# Course: DSC650 Big Data
# Week: 7
# Assignment Number: 7.1a


# In this part of the assignment, you will
# partition a dataset using different strategies.
# You will use the routes.parquet dataset you
# created in a previous assignment. For this
# dataset, the key for each route will be the
# three-letter source airport code concatenated
# with the three-letter destination airport code
# and the two-letter airline. For instance, a
# route from Omaha Eppley Airfield (OMA) to Denver
# International Airport (DEN) on American Airlines
# (AA) has a key of OMADENAA.
#
# Start by loading the dataset from the previous
# assignment using Pandas's read_parquet method.
# Next, add the concatenated key then using Panda's
# apply method to create a new column called key. For
# this part of the example, we will create 16 partitions
# so that we can compare it to the partitions we create
# from hashed keys in the next part of the assignment.
# The partitions are determined by the first letter of
# the composite key using the following partitions.


import os
import shutil
import json
from pathlib import Path
import gzip
import pandas as pd

In [2]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_dir = results_dir.joinpath('kv')
results_dir.mkdir(parents=True, exist_ok=True)
if os.path.exists(kv_dir):
    shutil.rmtree(kv_dir)
os.makedirs(kv_dir)

In [3]:
# 1): Load the dataset
def read_jsonl_data():
    src_data_path = 'data/routes.jsonl.gz'
    with gzip.open(src_data_path, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]

    return records

In [4]:
# 2): Flatten the records:
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    return flat_record


def create_flattened_dataset():
    records = read_jsonl_data()
    return pd.DataFrame.from_records([flatten_record(record) for record in records])


df = create_flattened_dataset()

In [5]:
# 3): Add key column:
df['key'] = df['src_airport_iata'].map(str) + \
            df['dst_airport_iata'].map(str) + \
            df['airline_iata'].map(str)

In [6]:
# 4): Add kv_key column:
df['kv_key'] = df['key'].astype(str).str[0]

In [7]:
# 5): Update kv_key column for partitioning
df['kv_key'].mask(df['kv_key'] == 'C', 'C-D', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'D', 'C-D', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'E', 'E-F', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'F', 'E-F', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'G', 'G-H', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'H', 'G-H', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'I', 'I-J', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'J', 'I-J', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'K', 'K-L', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'L', 'K-L', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'O', 'O-P', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'P', 'O-P', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'Q', 'Q-R', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'R', 'Q-R', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'S', 'S-T', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'T', 'S-T', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'W', 'W-X', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'X', 'W-X', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'Y', 'Y-Z', inplace=True)
df['kv_key'].mask(df['kv_key'] == 'Z', 'Y-Z', inplace=True)


# Verify the updated kv_key column data:
print(df)
pd.set_option('display.max_columns', None)
print(df.head())

       airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  Apache   
67661               19016        Apache Air                  Apache   
67662               19016        Apache Air                  Apache   

      airline_iata airline_icao airline_callsign airline_country  \
0               2B          ARD       AEROCONDOR        Portugal   
1          

In [8]:
# 6): Partitions (I'm just using as reference)
# partitions = (
#     ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
#     ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
#     ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
#     ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
# )

In [10]:
# 7): Write flattened df to parquet:
def write_parquet_file():
    pq_flat_path = results_dir.joinpath('routes-flattened.parquet')
    df.to_parquet(pq_flat_path)


write_parquet_file()

In [11]:
# 8): Write flattened df to partitioned parquet:
def write_partitioned_parquet_files():
    kv_key_path = kv_dir.joinpath()
    df.to_parquet(kv_key_path, partition_cols=['kv_key'])


write_partitioned_parquet_files()